In [2]:
import sys

sys.path.append('/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/python')

In [3]:
import jax
from flax.linen import relu, softmax, sigmoid
from nn.jax.multi_layer_dense import MultiLayerDense

In [4]:
predict_fn = MultiLayerDense(features=[16, 32, 7], activations=[relu, relu, softmax ])

In [5]:
predict_fn_params = predict_fn.init(jax.random.PRNGKey(0), jax.random.uniform(jax.random.PRNGKey(0), (7,)))

In [6]:
def loss(parameters, embedding, label):
    prediction = predict_fn.apply(parameters, embedding)
    label_probs = prediction * label + (1 - prediction) * (1 - label)
    return -jax.numpy.sum(jax.numpy.log(label_probs))
def batch_loss(parameters, batch_embeddings, batch_labels):
    losses = jax.vmap(loss, (None, 0, 0))(parameters, batch_embeddings, batch_labels)
    return jax.numpy.sum(losses)

In [7]:
labels = jax.numpy.eye(7)

In [8]:
logits = jax.random.uniform(jax.random.PRNGKey(0), (7,7))

In [54]:
for i in range(100):
    

    primals, f_vjp = jax.vjp(batch_loss, predict_fn_params, logits, labels)

    print(primals)

    delta = jax.grad(batch_loss)(predict_fn_params, logits, labels)

    predict_fn_params = jax.tree_multimap(lambda x, y: jax.numpy.asarray(x - (y * 0.03)), predict_fn_params, delta)

23.153784
23.079899
23.007263
22.935833
22.86558
22.796467
22.72847
22.66156
22.595722
22.530933
22.467182
22.404453
22.342735
22.28202
22.222298
22.163568
22.105824
22.049057
21.993273
21.938465
21.884636
21.83178
21.7799
21.728992
21.679066
21.630112
21.58213
21.535128
21.489098
21.444038
21.399954
21.356838
21.314692
21.273506
21.233284
21.19402
21.155706
21.11834
21.081911
21.04642
21.011852
20.978203
20.945461
20.91362
20.882668
20.852596
20.823393
20.795044
20.76754
20.740868
20.715014
20.689968
20.66571
20.642227
20.619513
20.597542
20.576305
20.55579
20.535976
20.51685
20.498398
20.4806
20.463446
20.446915
20.431
20.415676
20.400932
20.386753
20.373127
20.360031
20.347454
20.33538
20.3238
20.312689
20.30204
20.291838
20.282066
20.272713
20.263767
20.255207
20.24703
20.239216
20.231754
20.224632
20.217834
20.211355
20.20518
20.199295
20.193691
20.188358
20.183285
20.178461
20.173874
20.169518
20.165379
20.161453
20.157726
20.154192
20.15084
20.147665


In [56]:
a = (jax.numpy.ones(7), jax.numpy.ones(7), jax.numpy.ones(7))

In [62]:
b = jax.numpy.vstack(a)

In [72]:
s = jax.numpy.sum(b, axis=0)

In [75]:
a = [(1,1), (2,2), (3,3)]

In [81]:
list(map(lambda x: x[0], a))

[1, 2, 3]

In [101]:
def f(x):
    return jax.lax.log(x) * x**2

In [103]:
jax.jacfwd(f)(jax.numpy.array([1.,2.,3.]))

DeviceArray([[1.       , 0.       , 0.       ],
             [0.       , 4.7725887, 0.       ],
             [0.       , 0.       , 9.591674 ]], dtype=float32)

In [89]:
f(3.)

DeviceArray(9.88751, dtype=float32, weak_type=True)